# Applied Data Science Week 3 Project

## 1. Scrape the wikipedia page to get postal codes of Canada into a data frame
> *https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M*

In [2]:
!pip install beautifulsoup4
# !pip install wget

     |████████████████████████████████| 122kB 7.4MB/s eta 0:00:01


In [3]:
from bs4 import BeautifulSoup
import pandas as pd
# import wget
import requests

In [4]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = requests.get(url)

In [5]:
parsed_html = BeautifulSoup(page.content, 'html.parser')
table=parsed_html.body.find('table', attrs={'class':'wikitable'})

# define the dataframe columns
column_names = ['PostalCode', 'Borough', 'Neighborhood'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

# print(table.tbody.children)
for trow in table.tbody.children:
    td = trow.extract().find_all("td")
    if (len(td) > 1):
        neighborhoods = neighborhoods.append({'PostalCode': td[0].string.strip("\n"),
                                            'Borough': td[1].string.strip("\n"),
                                          'Neighborhood': td[2].string.strip("\n")}, ignore_index=True)

neighborhoods.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


### Now clean up the dataframe

In [6]:
neighborhoods = neighborhoods[neighborhoods.Borough != 'Not assigned']

print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

neighborhoods.head()

The dataframe has 10 boroughs and 103 neighborhoods.


,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### Verify there are no "Not assigned" cells in Neighborhood

In [7]:
df = neighborhoods
df[df.Neighborhood == "Not assigned"].head()

,PostalCode,Borough,Neighborhood


In [128]:
# df = neighborhoods.groupby(["PostalCode"]).count()
# df = df.reset_index()
# df.head()

# # df.describe()
# # df[df.PostalCode == "M1A"]

In [8]:

print("The shape of the dataframe is {}".format(df.shape))

The shape of the dataframe is (103, 3)


## 2. Get the lat, long for neighborhoods using geocoder data

In [15]:
response = requests.get("https://cocl.us/Geospatial_data")
open('geodata.csv', 'w').write(response.text)

2891

In [19]:
geodata = pd.read_csv("geodata.csv")
geodata.rename(columns={"Postal Code": "PostalCode"}, inplace=True)
geodata.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Merge both data frames

In [23]:
df = pd.merge(neighborhoods, geodata, on="PostalCode")
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


## 3. Now cluster the neighborhoods in Toronto and show on map